In [1]:
'''Autores: Daniel Esteban Cifuentes Cossio
            Cristian Cano Osorio
   Asesor:  Pedro Atencio Ortiz'''

#Carga de librerias necesarias

import numpy as np
import pickle as pk

from keras.models import Model
from keras.applications import vgg16
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.preprocessing import image

from skimage.io import imread, imshow
from skimage.transform import resize
import cv2

from PIL import Image

from os import listdir
from os.path import isfile



C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
'''
Pedro Atencio
2018
Este ejemplo extrae la activacion de la pernultima capa
de una red neuronal profunda pre-entrenada. En este caso
VGG16. 
En este codigo se utilizo Keras 2.1.6, Tensorflow 1.8.0
'''
#1. Cargamos el modelo completo
vgg16_model = vgg16.VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

#2. Creamos el modelo intermedio (penultima capa) a partir del modelo completo
vgg16_int_model = Model(inputs=vgg16_model.input, outputs = vgg16_model.get_layer('fc2').output)


#Función la cual carga una imagen, se pre-procesa y se obtiene el vector de activación

def obtenerActivacion(img):

    #Cargamos la imagen y la pre-procesamos
    x = resize(img, output_shape=(224, 224))
    x = np.expand_dims(x, axis=0)
    x = x.astype(np.float64)*255.0
    x = x = preprocess_input(x)

    #Enviamos la imagen al modelo y obtenemos el vector de activacion de la penultima capa
    deep_feature = vgg16_int_model.predict(x)
    return deep_feature
    

In [3]:
'''Método para traer la intersección de una imagen a color (3 canales)
y su respectiva mascara (1 canal)'''


def transformarImagen(rutaColor,rutaBlancoNegro):
        
    #Se lee las imagenenes
    imagenColor = imread(rutaColor)
    imagenBN = imread(rutaBlancoNegro)
    #se utiliza bitwise_and para obtener su intersección
    imcv = cv2.bitwise_and(imagenColor[:,:,0],imagenBN,1)
    imcv2 = cv2.bitwise_and(imagenColor[:,:,1],imagenBN,1)
    imcv3 = cv2.bitwise_and(imagenColor[:,:,2],imagenBN,1)
    
    #se convierte nuevamente a una imagén RGB (3 canales)
    rgbArray = np.zeros((480,640,3), 'uint8')
    rgbArray[..., 0] = imcv
    rgbArray[..., 1] = imcv2
    rgbArray[..., 2] = imcv3
    img = Image.fromarray(rgbArray)  
    image = np.asarray(img, dtype=np.uint8)

    return (image)


In [4]:
#Guarda una matriz en una ruta especifica
def saveMatrix(name, path, matrix):
    sv=path+name
    with open(sv,'wb') as fp:
        pk.dump(matrix,fp)

#Carga una matriz en una ruta especifica
def loadMatrix(name, path):
    sv=path+name
    with open(sv,'rb') as fp:
        return pk.load(fp)

In [5]:
#Obtiene la cantidad de archivos en una ruta especifica
def get_dataset_size(path):
    files = listdir(path)
    
    number_of_folders = 0
    for f in files:
        number_of_folders += 1
    
    return number_of_folders

In [6]:
'''Ejemplo de ejecución

Este ejemplo crea un archivo llamado prueba
el cual contiene una matriz de mx4096 donde m
es igual al número de imagenes procesadas, 
los datos para la matriz son los obtenidos por 
el modelo vgg16 en su penúltima capa'''


pathFolderRoot = "../Dataset/"
pathFolderResult = "../Results/"
pathImagesColor = pathFolderRoot+"/Color/A/tr01_cam00/"
pathImagesSegment = pathFolderRoot+"/Segmentado/A/tr01_cam00/"
    
imagesColor = listdir(pathImagesColor)
imagesSegment = listdir(pathImagesSegment)
matrixTotal = 0
i = 0

for img in imagesColor:
    if (int(img.split(".")[0]) > 169):
        ##se obtiene cada imagén
        i = i +1
        imgColor = imread(pathImagesColor+img)
        imgSegment = imread(pathImagesSegment+img)
        ##Se obtiene la intersección de las imagenes
        imgTransform = transformarImagen(pathImagesColor+img,pathImagesSegment+img)
        ##Se obtiene la activación
        vector = obtenerActivacion(imgTransform)
        ##se comienza a partir de la imagén que tambien tiene una mascara
        if(int(img.split(".")[0]) == 170):
            matrixTotal = vector
        else:
            matrixTotal = np.concatenate ((matrixTotal,vector),axis = 0)
            
saveMatrix('prueba',pathFolderResult,matrixTotal)

            


C:\Users\user\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [10]:
#Verificación de la matrix guardada
loadedMatrix = loadMatrix('prueba', pathFolderResult)
print ('tamaño',loadedMatrix.shape)
print (loadedMatrix)


tamaño (205, 4096)
[[0.         0.5770166  0.51340777 ... 0.         0.         1.2363524 ]
 [0.         0.8326682  0.5063835  ... 0.         0.         1.2805879 ]
 [0.         0.58754545 0.4720834  ... 0.         0.         1.5510166 ]
 ...
 [0.         0.41070285 0.21805173 ... 0.         0.         0.2684066 ]
 [0.         0.41070285 0.21805173 ... 0.         0.         0.2684066 ]
 [0.         0.41070285 0.21805173 ... 0.         0.         0.2684066 ]]
